In [1]:
env = 'staging'
pk_project = 153

import pandas as pd
import duckdb

import geovpylib.database as db
import geovpylib.utils as u
import geovpylib.sparql as sparql
import geovpylib.pks as pks
import geovpylib.graphs as graphs

db.connect(env, pk_project, execute=False)

Requests will not be executed
=== Setting STAGING environment ===
>> Connecting to PGSQL Database ... Connected!


## Fetch HLS data from Wikidata

In [ ]:
sparql.init('https://query.wikidata.org/sparql')